In [4]:
import os
import pandas as pd
import numpy as np
import google

ModuleNotFoundError: No module named 'google'

In [4]:
pip install dash

In [5]:
pip install dash-bootstrap-components

In [6]:
pip install dash_bootstrap_templates

In [7]:
pip install pandas fuzzywuzzy python-Levenshtein

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

ModuleNotFoundError: No module named 'google'

Helper Functions

In [11]:
def clean_up_stats_df(df):
    # Use first row as header
    df.columns = df.iloc[0].astype(str)
    df = df.drop(index=0).reset_index(drop=True)

    # Sanitize column names (handle duplicates like "..." etc.)
    seen = {}
    new_columns = []
    for col in df.columns:
        if col not in seen:
            seen[col] = 1
            new_columns.append(col)
        else:
            seen[col] += 1
            new_columns.append(f"{col}_{seen[col]}")
    df.columns = new_columns

    # Convert all columns except Name to float
    df["Name"] = df["Name"].astype(str)
    for col in df.columns:
        if col != "Name":
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

def clean_up_available_players_df(df):
  df.columns = ['Player', 'Overall Rating']
  for col in df.columns:
    if col != "Player":
        df[col] = pd.to_numeric(df[col], errors='coerce')
  return df

def clean_up_draft_df(df):
  df.columns = df.iloc[0]
  df = df.drop(index=0).reset_index(drop=True)
  df = df.rename(columns={df.columns[0]: "Round"}).set_index("Round")
  return df


In [12]:
draft_sh = gc.open('Season 24 Draft')

dataframes = {}
for sheet in draft_sh.worksheets():
    dataframes[sheet.title] = pd.DataFrame(sheet.get_all_values())

pg_df = clean_up_available_players_df(dataframes['PG'])
sg_df = clean_up_available_players_df(dataframes['SG'])
sf_df = clean_up_available_players_df(dataframes['SF'])
pf_df = clean_up_available_players_df(dataframes['PF'])
c_df = clean_up_available_players_df(dataframes['C'])
draft_sheet = clean_up_draft_df(dataframes['Draft Sheet'])

stats_sh = gc.open('Showcase Stats')

dataframes = {}
for sheet in stats_sh.worksheets():
    dataframes[sheet.title] = pd.DataFrame(sheet.get_all_values())

averages_df = clean_up_stats_df(dataframes['Averages'])
per_36_df = clean_up_stats_df(dataframes['Per 36'])